In [21]:
from MultiEurlexDataset import MultiEurlexDataset
from MultiEvalDataset import MultiEvalDataset
from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizerFast, TrainingArguments, Trainer, DataCollatorWithPadding, TextClassificationPipeline, EarlyStoppingCallback
from torch.utils.data import RandomSampler
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import numpy as np
import torch.nn as nn

In [2]:
import torch
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
#tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base', padding=False, trunaction=False)
#training_dataset = MultiEvalDataset(languages = ['hr'])
#lengths = []
#for i in range(len(training_dataset)):
#    lengths.append(len(training_dataset[i][0].split(' ')))
#for i in [10, 20, 30, 40, 50, 60, 70, 80, 90]:
#    print(np.percentile(lengths, i, axis=0))

In [3]:
device='cuda'
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base', padding=True, trunaction=True)
training_dataset = MultiEurlexDataset(languages = ['en'], tokenizer=tokenizer)
eval_dataset = MultiEurlexDataset(split='validation',languages = ['hr'], tokenizer=tokenizer)

Reusing dataset multi_eurlex (/home/mrajnovic/.cache/huggingface/datasets/multi_eurlex/all_languages/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)


  0%|          | 0/3 [00:00<?, ?it/s]

0.0% of dataset loaded
10.0% of dataset loaded
20.0% of dataset loaded
30.0% of dataset loaded
40.0% of dataset loaded
50.0% of dataset loaded
60.0% of dataset loaded
70.0% of dataset loaded
80.0% of dataset loaded
90.0% of dataset loaded
Loading dataset done


Reusing dataset multi_eurlex (/home/mrajnovic/.cache/huggingface/datasets/multi_eurlex/all_languages/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)


  0%|          | 0/3 [00:00<?, ?it/s]

0.0% of dataset loaded
10.0% of dataset loaded
20.0% of dataset loaded
30.0% of dataset loaded
40.0% of dataset loaded
50.0% of dataset loaded
60.0% of dataset loaded
70.0% of dataset loaded
80.0% of dataset loaded
90.0% of dataset loaded
Loading dataset done


In [4]:
test_dataset = MultiEurlexDataset(split='test',languages = ['hr'], tokenizer=tokenizer)

Reusing dataset multi_eurlex (/home/mrajnovic/.cache/huggingface/datasets/multi_eurlex/all_languages/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6)


  0%|          | 0/3 [00:00<?, ?it/s]

0.0% of dataset loaded
10.0% of dataset loaded
20.0% of dataset loaded
30.0% of dataset loaded
40.0% of dataset loaded
50.0% of dataset loaded
60.0% of dataset loaded
70.0% of dataset loaded
80.0% of dataset loaded
90.0% of dataset loaded
Loading dataset done


In [22]:
collator_fn = DataCollatorWithPadding(tokenizer,return_tensors="pt", padding="max_length", max_length=512)

In [23]:
metric = f1_score

def compute_metrics(eval_pred):
    label_pred, label_true = eval_pred
    sigmoid = nn.Sigmoid()
    label_pred=sigmoid(torch.tensor(label_pred)).numpy()
    label_pred[label_pred < 0.5] = 0
    label_pred[label_pred >= 0.5] = 1
    result={}
    result['f1'] = metric(label_true,label_pred, average='samples')
    result['classification_report'] = classification_report(label_true,label_pred)
    result['confusion_matrix'] = confusion_matrix(label_true, label_pred)
    return result

In [24]:
training_args = TrainingArguments(
    output_dir='./results/germanic',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_ratio=0.1,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_strategy='steps',
    logging_steps=1000,
    learning_rate=1e-5,
    save_strategy='epoch',
    evaluation_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    lr_scheduler_type='polynomial',
    log_level='info',
    dataloader_num_workers=8
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [25]:
model = XLMRobertaForSequenceClassification.from_pretrained('/home/mrajnovic/EuroVoc_project/results/slavic_croatian/checkpoint-72795', num_labels=21, problem_type="multi_label_classification", )

loading configuration file /home/mrajnovic/EuroVoc_project/results/slavic_croatian/checkpoint-72795/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LAB

In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=training_dataset,
    eval_dataset=eval_dataset,
    
    compute_metrics=compute_metrics,
    data_collator=collator_fn,
    #callbacks=[
    #    EarlyStoppingCallback(early_stopping_patience=5),
    #]
)

In [11]:
training_args.num_train_epochs

10

In [12]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, function_to_apply='sigmoid')

In [13]:
os.environ["WANDB_DISABLED"] = "true"
trainer.train()

/home/mrajnovic/miniconda3/envs/tar/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 55000
  Num Epochs = 10
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 68750


Epoch,Training Loss,Validation Loss


In [27]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 5000
  Batch size = 16


ValueError: multilabel-indicator is not supported

In [ ]:
model(input_ids=training_dataset[0]['input_ids'].unsqueeze(0).to(device),attention_mask=training_dataset[0]['attention_mask'].unsqueeze(0).to(device))

SequenceClassifierOutput(loss=None, logits=tensor([[-1.2594,  2.0396, -3.9613,  1.3616, -2.1823, -3.1136, -3.2287,  0.2411,
         -2.1513, -3.2607, -3.1584, -2.7092, -0.9786, -3.5652, -1.6449, -1.1268,
         -1.3294, -3.5044, -3.3167, -3.3110,  0.1766]], device='cuda:0',
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [20]:
print('              precision    recall  f1-score   support\n\n           0       0.60      0.75      0.67       774\n           1       0.78      0.70      0.74       653\n           2       0.85      0.85      0.85       974\n           3       0.92      0.71      0.80      2769\n           4       0.65      0.75      0.69       394\n           5       0.82      0.79      0.80      1086\n           6       0.85      0.88      0.86      1960\n           7       0.70      0.59      0.65       622\n           8       0.91      0.81      0.86       600\n           9       0.34      0.60      0.43       164\n          10       0.53      0.46      0.49       335\n          11       0.59      0.58      0.59       531\n          12       0.67      0.62      0.65       513\n          13       0.53      0.30      0.39       102\n          14       0.85      0.67      0.75       185\n          15       0.80      0.81      0.80      1347\n          16       0.36      0.21      0.27        57\n          17       0.96      0.95      0.95      1641\n          18       0.93      0.91      0.92      2208\n          19       0.64      0.59      0.62       444\n          20       0.65      0.59      0.62       610\n\n   micro avg       0.81      0.77      0.79     17969\n   macro avg       0.71      0.67      0.68     17969\nweighted avg       0.81      0.77      0.79     17969\n samples avg       0.83      0.80      0.79     17969\n')

              precision    recall  f1-score   support

           0       0.60      0.75      0.67       774
           1       0.78      0.70      0.74       653
           2       0.85      0.85      0.85       974
           3       0.92      0.71      0.80      2769
           4       0.65      0.75      0.69       394
           5       0.82      0.79      0.80      1086
           6       0.85      0.88      0.86      1960
           7       0.70      0.59      0.65       622
           8       0.91      0.81      0.86       600
           9       0.34      0.60      0.43       164
          10       0.53      0.46      0.49       335
          11       0.59      0.58      0.59       531
          12       0.67      0.62      0.65       513
          13       0.53      0.30      0.39       102
          14       0.85      0.67      0.75       185
          15       0.80      0.81      0.80      1347
          16       0.36      0.21      0.27        57
          17       0.96    